# Clase 1: Del texto al vector.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize

## Extracción de datos

In [ ]:
data = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [ ]:
data.target_names

In [ ]:
# Carga de los datos.

categories = ['alt.atheism', 'sci.space']
data = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
texts = data.data
labels = data.target
target_names = data.target_names

In [ ]:
# Plot general para los datos
def plot_representation(data_2d, labels, title):
    plt.figure(figsize=(10, 7))
    for label_idx in range(len(categories)):
        indices = np.where(labels == label_idx)
        plt.scatter(data_2d[indices, 0], data_2d[indices, 1], 
                    label=target_names[label_idx], alpha=0.7, edgecolors='w')
    
    plt.title(f"Representación: {title}", fontsize=14)
    plt.xlabel("Dimensión Latente 1")
    plt.ylabel("Dimensión Latente 2")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()

## Bag of Words.

In [ ]:
bow_vectorizer = CountVectorizer(stop_words='english')
bow_matrix = bow_vectorizer.fit_transform(texts)

In [ ]:
bow_matrix.shape

In [ ]:
bow_2d = TruncatedSVD(n_components=2).fit_transform(bow_matrix)
bow_2d.shape

In [ ]:
plot_representation(bow_2d,labels,"Bag of words")

## TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

In [ ]:
tfidf_matrix.shape

In [ ]:
tfidf_2d = TruncatedSVD(n_components=2).fit_transform(tfidf_matrix)
tfidf_2d

In [ ]:
plot_representation(tfidf_2d,labels,"Representacion: TF-IDF")

## LSA

In [ ]:
# Es practicamente lo que hicimos arriba
lsa_model = TruncatedSVD(n_components=100)
lsa_low_dim = lsa_model.fit_transform(tfidf_matrix)
lsa_2d = TruncatedSVD(n_components=2).fit_transform(lsa_low_dim)

In [ ]:
plot_representation(lsa_2d,labels,"Representacion: TF-IDF")

## W2V

In [ ]:
# Tokenizacion de texto
nltk.download('punkt')
tokenized_texts = [word_tokenize(doc.lower()) for doc in texts]

In [ ]:
word_tokenize(texts[0])

In [ ]:
def get_doc_vector(w2v_model,tokens):
    vecs = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(512)

In [ ]:
w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=512, window=5, min_count=2, workers=4)

In [ ]:
w2v_matrix = np.array([get_doc_vector(w2v_model,t) for t in tokenized_texts])
w2v_matrix.shape

In [ ]:
w2v_2d = TruncatedSVD(n_components=2).fit_transform(w2v_matrix)
w2v_2d

In [ ]:
plot_representation(w2v_2d,labels,"Representacion: W2V")